In [1]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def tanh(x):
    return np.tanh(x)

def sigmoid_derivative(x):
    return sigmoid(x) * (1 - sigmoid(x))

def tanh_derivative(x):
    return 1 - np.tanh(x)**2

def mse_loss(y_pred, y_true):
    return ((y_pred - y_true) ** 2).mean()

def mse_loss_derivative(y_pred, y_true):
    return 2 * (y_pred - y_true) / y_true.size

class GRUCell:
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.learning_rate = learning_rate
        # Initialize weights
        self.init_weights()
        
    def init_weights(self):
        self.Wz = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.Uz = np.random.randn(self.hidden_size, self.hidden_size) * 0.1
        self.bz = np.zeros((self.hidden_size, 1))
        
        self.Wr = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.Ur = np.random.randn(self.hidden_size, self.hidden_size) * 0.1
        self.br = np.zeros((self.hidden_size, 1))
        
        self.Wh = np.random.randn(self.hidden_size, self.input_size) * 0.1
        self.Uh = np.random.randn(self.hidden_size, self.hidden_size) * 0.1
        self.bh = np.zeros((self.hidden_size, 1))
        
        self.Wy = np.random.randn(self.output_size, self.hidden_size) * 0.1
        self.by = np.zeros((self.output_size, 1))
        
    def forward(self, x, h_prev):
        # Store values for backpropagation
        self.x, self.h_prev = x, h_prev
        
        # Update gate
        self.z = sigmoid(np.dot(self.Wz, x) + np.dot(self.Uz, h_prev) + self.bz)
        
        # Reset gate
        self.r = sigmoid(np.dot(self.Wr, x) + np.dot(self.Ur, h_prev) + self.br)
        
        # Candidate hidden state
        self.h_tilde = tanh(np.dot(self.Wh, x) + np.dot(self.Uh, self.r * h_prev) + self.bh)
        
        # Final hidden state
        h_next = self.z * h_prev + (1 - self.z) * self.h_tilde
        
        # Output
        y_pred = np.dot(self.Wy, h_next) + self.by
        
        return y_pred, h_next

    def backward(self, d_y_pred, d_h_next):
        # Gradient of the output layer
        d_Wy = np.dot(d_y_pred, self.h_prev.T)
        d_by = d_y_pred.sum(axis=1, keepdims=True)
        d_h_next += np.dot(self.Wy.T, d_y_pred)
        
        # Derivative of final hidden state
        d_z = d_h_next * (self.h_prev - self.h_tilde)
        d_h_prev = d_h_next * self.z
        d_h_tilde = d_h_next * (1 - self.z)
        
        # Derivatives of the gates
        d_h_tilde_raw = d_h_tilde * tanh_derivative(self.h_tilde)
        d_r = np.dot(self.Uh.T, d_h_tilde_raw) * self.h_prev
        d_h_prev += np.dot(self.Uh.T, d_h_tilde_raw) * self.r
        
        # Update weights and biases
        self.Wh -= self.learning_rate * np.dot(d_h_tilde_raw, self.x.T)
        self.Uh -= self.learning_rate * np.dot(d_h_tilde_raw, (self.r * self.h_prev).T)
        self.bh -= self.learning_rate * d_h_tilde_raw.sum(axis=1, keepdims=True)
        
        self.Wr -= self.learning_rate * np.dot(d_r * sigmoid_derivative(self.r), self.x.T)
        self.Ur -= self.learning_rate * np.dot(d_r * sigmoid_derivative(self.r), self.h_prev.T)
        self.br -= self.learning_rate * (d_r * sigmoid_derivative(self.r)).sum(axis=1, keepdims=True)
        
        self.Wz -= self.learning_rate * np.dot(d_z * sigmoid_derivative(self.z), self.x.T)
        self.Uz -= self.learning_rate * np.dot(d_z * sigmoid_derivative(self.z), self.h_prev.T)
        self.bz -= self.learning_rate * (d_z * sigmoid_derivative(self.z)).sum(axis=1, keepdims=True)
        
        self.Wy -= self.learning_rate * d_Wy
        self.by -= self.learning_rate * d_by

        return d_h_prev

    def train(self, inputs, targets, epochs):
        h_prev = np.zeros((self.hidden_size, 1))
        
        for epoch in range(epochs):
            loss = 0
            for t in range(len(inputs)):
                x = inputs[t]
                y_true = targets[t]
                
                # Forward pass
                y_pred, h_next = self.forward(x, h_prev)
                
                # Calculate loss (for monitoring)
                loss += mse_loss(y_pred, y_true)
                
                # Backpropagate error
                d_loss = mse_loss_derivative(y_pred, y_true)
                d_h_next = self.backward(d_loss, h_next)
                h_prev = h_next  # update state
                
            loss /= len(inputs)
            print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss}')

    def predict(self, inputs):
        h_prev = np.zeros((self.hidden_size, 1))
        predictions = []
        for x in inputs:
            y_pred, h_prev = self.forward(x, h_prev)
            predictions.append(y_pred)
        return predictions

In [10]:
input_size = 10
hidden_size = 30
output_size = 5  
epochs = 10
learning_rate = 0.01

# Example synthetic data
inputs = [np.random.randn(input_size, 1) for _ in range(1000)]
targets = [np.random.randn(output_size, 1) for _ in range(1000)]
gru_cell = GRUCell(input_size, hidden_size, output_size, learning_rate)
gru_cell.train(inputs, targets, epochs)

Epoch 1/10, Loss: 1.008868433911287
Epoch 2/10, Loss: 1.0042903955863998
Epoch 3/10, Loss: 1.0033678354570383
Epoch 4/10, Loss: 1.0030373447767569
Epoch 5/10, Loss: 1.0029713099192374
Epoch 6/10, Loss: 1.0030191191644553
Epoch 7/10, Loss: 1.0031149145007352
Epoch 8/10, Loss: 1.0032299425360844
Epoch 9/10, Loss: 1.0033514089245317
Epoch 10/10, Loss: 1.0034735823205183


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
import tensorflow as tf 
import regex as re

def file_to_sentence_list(file_path): 
	with open(file_path, 'r') as file: 
		text = file.read() 

	# Splitting the text into sentences using 
	# delimiters like '.', '?', and '!' 
	sentences = [sentence.strip() for sentence in re.split( 
		r'(?<=[.!?])\s+', text) if sentence.strip()] 

	return sentences 

file_path = 'pizza.txt'
text_data = file_to_sentence_list(file_path) 

# Tokenize the text data 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(text_data) 
total_words = len(tokenizer.word_index) + 1

# Create input sequences 
input_sequences = [] 
for line in text_data: 
	token_list = tokenizer.texts_to_sequences([line])[0] 
	for i in range(1, len(token_list)): 
		n_gram_sequence = token_list[:i+1] 
		input_sequences.append(n_gram_sequence) 

# Pad sequences and split into predictors and label 
max_sequence_len = max([len(seq) for seq in input_sequences]) 
input_sequences = np.array(pad_sequences( 
	input_sequences, maxlen=max_sequence_len, padding='pre')) 
X, y = input_sequences[:, :-1], input_sequences[:, -1] 

# Convert target data to one-hot encoding 
y = tf.keras.utils.to_categorical(y, num_classes=total_words) 

In [4]:
inputs = np.array(X)
inputs = np.array([input.reshape(39, 1) for input in inputs])

targets = np.array(y)
targets = np.array([target.reshape(687, 1) for target in targets])

input_size = 39
hidden_size = 39
output_size = 687
epochs = 10
learning_rate = 0.01

In [5]:
gru_cell = GRUCell(input_size, hidden_size, output_size, learning_rate)
gru_cell.train(inputs, targets, epochs)

C:\Users\K-Gen\AppData\Local\Temp\ipykernel_9392\3916400805.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch 1/10, Loss: 0.39284285335721997
Epoch 2/10, Loss: 0.3734499227875016
Epoch 3/10, Loss: 0.35504859552657614
Epoch 4/10, Loss: 0.3378876641175672
Epoch 5/10, Loss: 0.32141977606192806
Epoch 6/10, Loss: 0.3052129036477114
Epoch 7/10, Loss: 0.28983460579356096
Epoch 8/10, Loss: 0.2754135228362305
Epoch 9/10, Loss: 0.2620004689130761
Epoch 10/10, Loss: 0.24711975113518647


In [9]:
predicted_probs = gru_cell.predict(X[10]) 
val = predicted_probs[0]
val.shape

C:\Users\K-Gen\AppData\Local\Temp\ipykernel_9392\3916400805.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


(687, 39)